In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import seaborn as sns
from pathlib import Path 

In [2]:
df= pd.read_csv(r"D:\Internal Project\I80_1st15minutes_complete\I80_1st15minutes_complete.csv")

In [3]:
df

,Unnamed: 0,Vehicle.ID,Frame.ID,Lane,Local.Y,svel,sacc,Vehicle.Length,Vehicle.class,Following.Vehicle.ID,...,frspacing,entryLane,entry,LCType,dV,svelkm,PrecVehVelkm,svelkm.level,PrecVehVelkm.level,dA
0,1,1,147,2,50.08315,2.84104,1.46320,4.3591,2,-1,...,NaN,2,MainLine,CFonly,NaN,10.227744,NaN,"(10,15]",NaN,NaN
1,2,1,148,2,50.38612,3.02972,1.88680,4.3591,2,-1,...,NaN,2,MainLine,CFonly,NaN,10.906992,NaN,"(10,15]",NaN,NaN
2,3,1,149,2,50.69421,3.08086,0.51134,4.3591,2,-1,...,NaN,2,MainLine,CFonly,NaN,11.091096,NaN,"(10,15]",NaN,NaN
3,4,1,150,2,51.00658,3.12377,0.42912,4.3591,2,-1,...,NaN,2,MainLine,CFonly,NaN,11.245572,NaN,"(10,15]",NaN,NaN
4,5,1,151,2,51.32203,3.15445,0.30681,4.3591,2,-1,...,NaN,2,MainLine,CFonly,NaN,11.356020,NaN,"(10,15]",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059016,1059017,3366,3198,1,440.79796,18.61708,0.18434,5.1212,2,978,...,NaN,1,MainLine,CFonly,NaN,67.021488,NaN,"(65,70]",NaN,NaN
1059017,1059018,3366,3199,1,442.66070,18.62743,0.10346,5.1212,2,978,...,NaN,1,MainLine,CFonly,NaN,67.058748,NaN,"(65,70]",NaN,NaN
1059018,1059019,3366,3200,1,444.52374,18.63045,0.03020,5.1212,2,978,...,NaN,1,MainLine,CFonly,NaN,67.069620,NaN,"(65,70]",NaN,NaN
1059019,1059020,3366,3201,1,446.38651,18.62770,-0.02746,5.1212,2,978,...,NaN,1,MainLine,CFonly,NaN,67.059720,NaN,"(65,70]",NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1059021 entries, 0 to 1059020
Data columns (total 38 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   Unnamed: 0            1059021 non-null  int64  
 1   Vehicle.ID            1059021 non-null  int64  
 2   Frame.ID              1059021 non-null  int64  
 3   Lane                  1059021 non-null  int64  
 4   Local.Y               1059021 non-null  float64
 5   svel                  1059021 non-null  float64
 6   sacc                  1059021 non-null  float64
 7   Vehicle.Length        1059021 non-null  float64
 8   Vehicle.class         1059021 non-null  int64  
 9   Following.Vehicle.ID  1059021 non-null  int64  
 10  Preceding.Vehicle.ID  1059021 non-null  int64  
 11  Vehicle.Width.ft      1059021 non-null  float64
 12  PrecVehClass          992286 non-null   float64
 13  FollVehClass          992007 non-null   float64
 14  PrecVehLength         992286 non-n

In [5]:
df = df[df["LCType"]=="CFonly"]

In [6]:
df['LCType'].unique()

array(['CFonly'], dtype=object)

In [4]:
df['Lane'].unique()

array([  2,   1,   5,   6,   4,   3,   7, 999], dtype=int64)

In [5]:
#selecting lanes.

Lanes = [1,2,3,4]

df = df[df['Lane'].isin(Lanes)]

In [6]:
df = df[df["Preceding.Vehicle.ID"] != -1]

In [7]:
df

,Unnamed: 0,Vehicle.ID,Frame.ID,Lane,Local.Y,svel,sacc,Vehicle.Length,Vehicle.class,Following.Vehicle.ID,...,frspacing,entryLane,entry,LCType,dV,svelkm,PrecVehVelkm,svelkm.level,PrecVehVelkm.level,dA
334,335,1,481,2,173.27443,6.94512,-0.96402,4.3591,2,11,...,2.16271,2,MainLine,CFonly,-2.06896,25.002432,32.450688,"(25,30]","(30,35]",0.65758
335,336,1,482,2,173.95693,6.82504,-1.20074,4.3591,2,11,...,2.36948,2,MainLine,CFonly,-2.06769,24.570144,32.013828,"(20,25]","(30,35]",0.01280
336,337,1,483,2,174.62545,6.68521,-1.39833,4.3591,2,11,...,2.58190,2,MainLine,CFonly,-2.12414,24.066756,31.713660,"(20,25]","(30,35]",-0.56450
337,338,1,484,2,175.27933,6.53877,-1.46441,4.3591,2,11,...,2.80349,2,MainLine,CFonly,-2.21593,23.539572,31.516920,"(20,25]","(30,35]",-0.91793
338,339,1,485,2,175.91971,6.40379,-1.34976,4.3591,2,11,...,3.03503,2,MainLine,CFonly,-2.31541,23.053644,31.389120,"(20,25]","(30,35]",-0.99473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058995,1058996,3366,3177,1,401.98395,19.22258,-1.47646,5.1212,2,978,...,33.60179,1,MainLine,CFonly,0.33214,69.201288,68.005584,"(65,70]","(65,70]",-2.33206
1058996,1058997,3366,3178,1,403.88877,19.04817,-1.74406,5.1212,2,978,...,33.60131,1,MainLine,CFonly,0.00470,68.573412,68.556492,"(65,70]","(65,70]",-3.27433
1058997,1058998,3366,3179,1,405.77437,18.85603,-1.92143,5.1212,2,978,...,33.63930,1,MainLine,CFonly,-0.37988,67.881708,69.249276,"(65,70]","(65,70]",-3.84586
1058998,1058999,3366,3180,1,407.64071,18.66341,-1.92618,5.1212,2,978,...,33.71659,1,MainLine,CFonly,-0.77285,67.188276,69.970536,"(65,70]","(65,70]",-3.92965


In [8]:
df[['Lane','Vehicle.ID']]

,Lane,Vehicle.ID
334,2,1
335,2,1
336,2,1
337,2,1
338,2,1
...,...,...
1058995,1,3366
1058996,1,3366
1058997,1,3366
1058998,1,3366


In [9]:
df.shape

(615679, 38)

In [10]:
df["Vehicle.ID"]=df["Vehicle.ID"].astype(str)
df["Preceding.Vehicle.ID"]=df["Preceding.Vehicle.ID"].astype(str)
df["LF_pairs"] = df["Vehicle.ID"]+ "_"+df["Preceding.Vehicle.ID"]

In [11]:
df["Vehicle.class"]=df["Vehicle.class"].astype(str)
df["PrecVehClass"]=df["PrecVehClass"].astype(str)
df["Veh.cls.pairs"] = df["Vehicle.class"]+ "_"+df["PrecVehClass"]

In [12]:
df["Time.per.frame"] = 0.1

In [13]:
df_grp = df.groupby(["LF_pairs"])
df_grp.groups

{'1001_994': [269127, 269128, 269129, 269130, 269131, 269132, 269133, 269134, 269135, 269136, 269137, 269138, 269139, 269140, 269141, 269142, 269143, 269144, 269145, 269146, 269147, 269148, 269149, 269150, 269151, 269152, 269153, 269154, 269155, 269156, 269157, 269158, 269159, 269160, 269161, 269162, 269163, 269164, 269165, 269166, 269167, 269168, 269169, 269170, 269171, 269172, 269173, 269174, 269175, 269176, 269177, 269178, 269179, 269180, 269181, 269182, 269183, 269184, 269185, 269186, 269187, 269188, 269189, 269190, 269191, 269192, 269193, 269194, 269195, 269196, 269197, 269198, 269199, 269200, 269201, 269202, 269203, 269204, 269205, 269206, 269207, 269208, 269209, 269210, 269211, 269212, 269213, 269214, 269215, 269216, 269217, 269218, 269219, 269220, 269221, 269222, 269223, 269224, 269225, 269226, ...], '1003_996': [269708, 269709, 269710, 269711, 269712, 269713, 269714, 269715, 269716, 269717, 269718, 269719, 269720, 269721, 269722, 269723, 269724, 269725, 269726, 269727, 269728,

In [14]:
#Time duration per pair of each time frame.
df['Time_duration']= df.groupby(["LF_pairs"], as_index=False)["Frame.ID"].cumcount()*0.1


In [15]:

new_column = df.groupby(["LF_pairs"], as_index=False).apply(lambda x : len(x["Frame.ID"]) * 0.1 )
df = pd.merge(df, new_column, how="inner", on=["LF_pairs"])


In [16]:
df["nextframeAcc"]=df.groupby(["LF_pairs"],as_index=False)["sacc"].shift(-1)

In [17]:
df["nextframesvel"]=df.groupby(["LF_pairs"],as_index=False)["svel"].shift(-1)

In [18]:
df["nextframeposition"]=df.groupby(["LF_pairs"],as_index=False)["Local.Y"].shift(-1)

In [19]:
df

,Unnamed: 0,Vehicle.ID,Frame.ID,Lane,Local.Y,svel,sacc,Vehicle.Length,Vehicle.class,Following.Vehicle.ID,...,PrecVehVelkm.level,dA,LF_pairs,Veh.cls.pairs,Time.per.frame,Time_duration,None,nextframeAcc,nextframesvel,nextframeposition
0,335,1,481,2,173.27443,6.94512,-0.96402,4.3591,2,11,...,"(30,35]",0.65758,1_3355,2_2.0,0.1,0.0,8.2,-1.20074,6.82504,173.95693
1,336,1,482,2,173.95693,6.82504,-1.20074,4.3591,2,11,...,"(30,35]",0.01280,1_3355,2_2.0,0.1,0.1,8.2,-1.39833,6.68521,174.62545
2,337,1,483,2,174.62545,6.68521,-1.39833,4.3591,2,11,...,"(30,35]",-0.56450,1_3355,2_2.0,0.1,0.2,8.2,-1.46441,6.53877,175.27933
3,338,1,484,2,175.27933,6.53877,-1.46441,4.3591,2,11,...,"(30,35]",-0.91793,1_3355,2_2.0,0.1,0.3,8.2,-1.34976,6.40379,175.91971
4,339,1,485,2,175.91971,6.40379,-1.34976,4.3591,2,11,...,"(30,35]",-0.99473,1_3355,2_2.0,0.1,0.4,8.2,-1.08862,6.29493,176.54920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615674,1058996,3366,3177,1,401.98395,19.22258,-1.47646,5.1212,2,978,...,"(65,70]",-2.33206,3366_977,2_2.0,0.1,21.8,22.3,-1.74406,19.04817,403.88877
615675,1058997,3366,3178,1,403.88877,19.04817,-1.74406,5.1212,2,978,...,"(65,70]",-3.27433,3366_977,2_2.0,0.1,21.9,22.3,-1.92143,18.85603,405.77437
615676,1058998,3366,3179,1,405.77437,18.85603,-1.92143,5.1212,2,978,...,"(65,70]",-3.84586,3366_977,2_2.0,0.1,22.0,22.3,-1.92618,18.66341,407.64071
615677,1058999,3366,3180,1,407.64071,18.66341,-1.92618,5.1212,2,978,...,"(65,70]",-3.92965,3366_977,2_2.0,0.1,22.1,22.3,-1.59872,18.50354,409.49107


In [20]:
df.to_csv("D:\STUDY MATERIALS\SEMESTER-4\DAB 402 CAPSTONE PROJECT\Internal Project\i80_processed_lanes_4.csv")

In [21]:
df.shape

(615679, 46)

In [22]:
df["LF_pairs"].value_counts()

2923_2914    844
3000_2996    839
3031_3021    835
2939_2923    830
2893_2898    827
            ... 
763_746        1
869_850        1
1994_2075      1
1032_1057      1
1319_1316      1
Name: LF_pairs, Length: 2164, dtype: int64

## Modelling


In [38]:
df_new = pd.read_csv("D:\STUDY MATERIALS\SEMESTER-4\DAB 402 CAPSTONE PROJECT\Internal Project\i80_processed.csv")

df_g = df_new.groupby(["LF_pairs"])
df_new

,Unnamed: 0,Unnamed: 0.1,Vehicle.ID,Frame.ID,Lane,Local.Y,svel,sacc,Vehicle.Length,Vehicle.class,...,PrecVehVelkm.level,dA,LF_pairs,Veh.cls.pairs,Time.per.frame,Time_duration,Time_per_pair,nextframeAcc,nextframesvel,nextframeposition
0,0,335,1,481,2,173.27443,6.94512,-0.96402,4.3591,2,...,"(30,35]",0.65758,1_3355,2_2.0,0.1,0.0,8.2,-1.20074,6.82504,173.95693
1,1,336,1,482,2,173.95693,6.82504,-1.20074,4.3591,2,...,"(30,35]",0.01280,1_3355,2_2.0,0.1,0.1,8.2,-1.39833,6.68521,174.62545
2,2,337,1,483,2,174.62545,6.68521,-1.39833,4.3591,2,...,"(30,35]",-0.56450,1_3355,2_2.0,0.1,0.2,8.2,-1.46441,6.53877,175.27933
3,3,338,1,484,2,175.27933,6.53877,-1.46441,4.3591,2,...,"(30,35]",-0.91793,1_3355,2_2.0,0.1,0.3,8.2,-1.34976,6.40379,175.91971
4,4,339,1,485,2,175.91971,6.40379,-1.34976,4.3591,2,...,"(30,35]",-0.99473,1_3355,2_2.0,0.1,0.4,8.2,-1.08862,6.29493,176.54920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353342,353342,1058996,3366,3177,1,401.98395,19.22258,-1.47646,5.1212,2,...,"(65,70]",-2.33206,3366_977,2_2.0,0.1,21.8,22.3,-1.74406,19.04817,403.88877
353343,353343,1058997,3366,3178,1,403.88877,19.04817,-1.74406,5.1212,2,...,"(65,70]",-3.27433,3366_977,2_2.0,0.1,21.9,22.3,-1.92143,18.85603,405.77437
353344,353344,1058998,3366,3179,1,405.77437,18.85603,-1.92143,5.1212,2,...,"(65,70]",-3.84586,3366_977,2_2.0,0.1,22.0,22.3,-1.92618,18.66341,407.64071
353345,353345,1058999,3366,3180,1,407.64071,18.66341,-1.92618,5.1212,2,...,"(65,70]",-3.92965,3366_977,2_2.0,0.1,22.1,22.3,-1.59872,18.50354,409.49107


In [39]:
for lf_pairs, data in df_new.groupby("LF_pairs"):

    data.to_csv("D:\STUDY MATERIALS\SEMESTER-4\DAB 402 CAPSTONE PROJECT\processed_csv\{}.csv".format(lf_pairs))


In [ ]:
df[df_new["Time_per_pair"]==71.5]

In [ ]:
df_pair = pd.read_csv(r"D:\STUDY MATERIALS\SEMESTER-4\DAB 402 CAPSTONE PROJECT\processed_csv\2777_2770.csv")
df_pair

In [ ]:
df_pair.columns

In [ ]:
df = df_pair[['svel','sacc','PrecVehVel','PrecVehAcc','frspacing','Time_duration']]
df

In [ ]:
sns.set_theme(style="darkgrid")
# Load an example dataset with long-form data
for data in df_new.groupby("LF_pairs"):
    sns.lineplot(x=df["svel"], y=df["Time_duration"],hue="sacc",data=data)
# Plot the responses for different events and regions


In [ ]:
sns.set_theme(style="darkgrid")

sns.lineplot(x="Time_duration", y="svel",data=df_pair)


In [ ]:
sns.set_theme(style="darkgrid")

sns.lineplot(x="Time_duration", y="PrecVehVel",data=df_pair)


In [ ]:
import pandas as pd
import seaborn as sns

my_dict = dict(x=Time_duration,y=svel,z=PrecVehVel)
data = df_pair(my_dict)
fig, ax = plt.subplots()
ax= sns.lineplot(x='Time_duration', y='svel', data=df_pair)
#ax1 = sns.lineplot(x='Time_duration', y='PrecVehVel', data=df_pair)

In [ ]:
#plotting graph for velocity between subject vehicle and preceding vehicle for the travelled duration.

sns.set(rc = {'figure.figsize':(15,8)})
sns.lineplot(data = df_pair[['svel','PrecVehVel']])
plt.legend(labels=["subjec_veh_vel","Pre_veh_velo"])

In [ ]:
#plotting graph for acceleration between subject vehicle and preceding vehicle for the travelled duration.
sns.set(rc = {'figure.figsize':(20,8)})
sns.lineplot(data = df_pair[['sacc','PrecVehAcc']])
plt.legend(labels=["subjec_veh_acce","Pre_veh_acce"])

# Time Series for acceleration column.


In [ ]:
df_t = df_pair[['Time_duration','sacc']]
df_t.set_index('Time_duration')

#df_t["last_frame"]

In [ ]:
df_t["previousframe"]=df_t[["sacc"]].shift(+1)
df_t["pr.frme.-2"]=df_t["sacc"].shift(+2)


In [ ]:
df_t

In [23]:
#processing i80_dataset.
df_i80 = pd.read_csv("D:\STUDY MATERIALS\SEMESTER-4\DAB 402 CAPSTONE PROJECT\Internal Project\i80_processed_lanes_4.csv")
df_i80

,Unnamed: 0.1,Unnamed: 0,Vehicle.ID,Frame.ID,Lane,Local.Y,svel,sacc,Vehicle.Length,Vehicle.class,...,PrecVehVelkm.level,dA,LF_pairs,Veh.cls.pairs,Time.per.frame,Time_duration,Unnamed: 43,nextframeAcc,nextframesvel,nextframeposition
0,0,335,1,481,2,173.27443,6.94512,-0.96402,4.3591,2,...,"(30,35]",0.65758,1_3355,2_2.0,0.1,0.0,8.2,-1.20074,6.82504,173.95693
1,1,336,1,482,2,173.95693,6.82504,-1.20074,4.3591,2,...,"(30,35]",0.01280,1_3355,2_2.0,0.1,0.1,8.2,-1.39833,6.68521,174.62545
2,2,337,1,483,2,174.62545,6.68521,-1.39833,4.3591,2,...,"(30,35]",-0.56450,1_3355,2_2.0,0.1,0.2,8.2,-1.46441,6.53877,175.27933
3,3,338,1,484,2,175.27933,6.53877,-1.46441,4.3591,2,...,"(30,35]",-0.91793,1_3355,2_2.0,0.1,0.3,8.2,-1.34976,6.40379,175.91971
4,4,339,1,485,2,175.91971,6.40379,-1.34976,4.3591,2,...,"(30,35]",-0.99473,1_3355,2_2.0,0.1,0.4,8.2,-1.08862,6.29493,176.54920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615674,615674,1058996,3366,3177,1,401.98395,19.22258,-1.47646,5.1212,2,...,"(65,70]",-2.33206,3366_977,2_2.0,0.1,21.8,22.3,-1.74406,19.04817,403.88877
615675,615675,1058997,3366,3178,1,403.88877,19.04817,-1.74406,5.1212,2,...,"(65,70]",-3.27433,3366_977,2_2.0,0.1,21.9,22.3,-1.92143,18.85603,405.77437
615676,615676,1058998,3366,3179,1,405.77437,18.85603,-1.92143,5.1212,2,...,"(65,70]",-3.84586,3366_977,2_2.0,0.1,22.0,22.3,-1.92618,18.66341,407.64071
615677,615677,1058999,3366,3180,1,407.64071,18.66341,-1.92618,5.1212,2,...,"(65,70]",-3.92965,3366_977,2_2.0,0.1,22.1,22.3,-1.59872,18.50354,409.49107


In [24]:
df_i80 = df_i80[df_i80["nextframeAcc"].notna()]
df_i80

,Unnamed: 0.1,Unnamed: 0,Vehicle.ID,Frame.ID,Lane,Local.Y,svel,sacc,Vehicle.Length,Vehicle.class,...,PrecVehVelkm.level,dA,LF_pairs,Veh.cls.pairs,Time.per.frame,Time_duration,Unnamed: 43,nextframeAcc,nextframesvel,nextframeposition
0,0,335,1,481,2,173.27443,6.94512,-0.96402,4.3591,2,...,"(30,35]",0.65758,1_3355,2_2.0,0.1,0.0,8.2,-1.20074,6.82504,173.95693
1,1,336,1,482,2,173.95693,6.82504,-1.20074,4.3591,2,...,"(30,35]",0.01280,1_3355,2_2.0,0.1,0.1,8.2,-1.39833,6.68521,174.62545
2,2,337,1,483,2,174.62545,6.68521,-1.39833,4.3591,2,...,"(30,35]",-0.56450,1_3355,2_2.0,0.1,0.2,8.2,-1.46441,6.53877,175.27933
3,3,338,1,484,2,175.27933,6.53877,-1.46441,4.3591,2,...,"(30,35]",-0.91793,1_3355,2_2.0,0.1,0.3,8.2,-1.34976,6.40379,175.91971
4,4,339,1,485,2,175.91971,6.40379,-1.34976,4.3591,2,...,"(30,35]",-0.99473,1_3355,2_2.0,0.1,0.4,8.2,-1.08862,6.29493,176.54920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615673,615673,1058995,3366,3176,1,400.06169,19.37023,-1.16084,5.1212,2,...,"(65,70]",-1.27048,3366_977,2_2.0,0.1,21.7,22.3,-1.47646,19.22258,401.98395
615674,615674,1058996,3366,3177,1,401.98395,19.22258,-1.47646,5.1212,2,...,"(65,70]",-2.33206,3366_977,2_2.0,0.1,21.8,22.3,-1.74406,19.04817,403.88877
615675,615675,1058997,3366,3178,1,403.88877,19.04817,-1.74406,5.1212,2,...,"(65,70]",-3.27433,3366_977,2_2.0,0.1,21.9,22.3,-1.92143,18.85603,405.77437
615676,615676,1058998,3366,3179,1,405.77437,18.85603,-1.92143,5.1212,2,...,"(65,70]",-3.84586,3366_977,2_2.0,0.1,22.0,22.3,-1.92618,18.66341,407.64071


In [25]:
df_i80 = df_i80[df_i80["PrecVehVel"].notna()]
df_i80

,Unnamed: 0.1,Unnamed: 0,Vehicle.ID,Frame.ID,Lane,Local.Y,svel,sacc,Vehicle.Length,Vehicle.class,...,PrecVehVelkm.level,dA,LF_pairs,Veh.cls.pairs,Time.per.frame,Time_duration,Unnamed: 43,nextframeAcc,nextframesvel,nextframeposition
0,0,335,1,481,2,173.27443,6.94512,-0.96402,4.3591,2,...,"(30,35]",0.65758,1_3355,2_2.0,0.1,0.0,8.2,-1.20074,6.82504,173.95693
1,1,336,1,482,2,173.95693,6.82504,-1.20074,4.3591,2,...,"(30,35]",0.01280,1_3355,2_2.0,0.1,0.1,8.2,-1.39833,6.68521,174.62545
2,2,337,1,483,2,174.62545,6.68521,-1.39833,4.3591,2,...,"(30,35]",-0.56450,1_3355,2_2.0,0.1,0.2,8.2,-1.46441,6.53877,175.27933
3,3,338,1,484,2,175.27933,6.53877,-1.46441,4.3591,2,...,"(30,35]",-0.91793,1_3355,2_2.0,0.1,0.3,8.2,-1.34976,6.40379,175.91971
4,4,339,1,485,2,175.91971,6.40379,-1.34976,4.3591,2,...,"(30,35]",-0.99473,1_3355,2_2.0,0.1,0.4,8.2,-1.08862,6.29493,176.54920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615673,615673,1058995,3366,3176,1,400.06169,19.37023,-1.16084,5.1212,2,...,"(65,70]",-1.27048,3366_977,2_2.0,0.1,21.7,22.3,-1.47646,19.22258,401.98395
615674,615674,1058996,3366,3177,1,401.98395,19.22258,-1.47646,5.1212,2,...,"(65,70]",-2.33206,3366_977,2_2.0,0.1,21.8,22.3,-1.74406,19.04817,403.88877
615675,615675,1058997,3366,3178,1,403.88877,19.04817,-1.74406,5.1212,2,...,"(65,70]",-3.27433,3366_977,2_2.0,0.1,21.9,22.3,-1.92143,18.85603,405.77437
615676,615676,1058998,3366,3179,1,405.77437,18.85603,-1.92143,5.1212,2,...,"(65,70]",-3.84586,3366_977,2_2.0,0.1,22.0,22.3,-1.92618,18.66341,407.64071


In [26]:
#Storing dataset to a new file

#df.to_csv("D:\Internal Project\i80_processed_lanes_4.csv")

In [27]:
df_i80.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Vehicle.ID', 'Frame.ID', 'Lane',
       'Local.Y', 'svel', 'sacc', 'Vehicle.Length', 'Vehicle.class',
       'Following.Vehicle.ID', 'Preceding.Vehicle.ID', 'Vehicle.Width.ft',
       'PrecVehClass', 'FollVehClass', 'PrecVehLength', 'FollVehLength',
       'lane.change', 'PrecVehVel', 'PrecVehAcc', 'FollVehVel', 'FollVehAcc',
       'Vehicle.type', 'PrecVehType', 'lanechng', 'PrecVehLocalY',
       'FollVehLocalY', 'spacing', 'headway', 'frspacing', 'entryLane',
       'entry', 'LCType', 'dV', 'svelkm', 'PrecVehVelkm', 'svelkm.level',
       'PrecVehVelkm.level', 'dA', 'LF_pairs', 'Veh.cls.pairs',
       'Time.per.frame', 'Time_duration', 'Unnamed: 43', 'nextframeAcc',
       'nextframesvel', 'nextframeposition'],
      dtype='object')

In [137]:
X.isnull().sum()

Time_duration    0
PrecVehVel       0
svel             0
frspacing        0
Local.Y          0
PrecVehLocalY    0
dtype: int64